In [46]:
import requests
import json
import pymongo
import ssl

MONGO_DETAILS = ""
client = pymongo.MongoClient(MONGO_DETAILS, ssl_cert_reqs=ssl.CERT_NONE)
database = client.nadlan

ERROR_MSG = "לא קיים בנתונים"
URL = "https://www.nadlan.gov.il/Nadlan.REST/Main/GetAssestAndDeals"


def asset_helper(asset):
    return {
        "Date": asset['DEALDATE'],
        "Address": asset['FULLADRESS'],
        "Gush": asset['GUSH'],
        "Type": asset['DEALNATUREDESCRIPTION'],
        "Num bdrms": asset['ASSETROOMNUM'],
        "Floor": asset['FLOORNO'],
        "Sq meter": asset['DEALNATURE'],
        "Price": asset["DEALAMOUNT"],
        "Building year": asset["BUILDINGYEAR"]
    }


def get_payload(city):
    payload_url = "https://www.nadlan.gov.il/Nadlan.REST/Main/GetDataByQuery"
    params = {
        'query': city
    }
    response = requests.get(url=payload_url, params=params)
    city_payload = json.loads(response.text)
    if city_payload['ResultLable'] == ERROR_MSG:
        print("Does not exist in the data")
        return False
    return city_payload


def insert_to_db(asset, city):
    city_collection = database.get_collection(city)
    city_collection.insert_one(asset_helper(asset))


def get_data_by_city(city):
    city_payload = get_payload(city)
    if city_payload:
        is_last_page = False
        page_num = 1
        while not is_last_page:
            is_last_page = write_data(city, page_num, city_payload)
            page_num += 1


def write_data(city, page_num, city_payload):
    asset_data = get_asset_data(page_num, city_payload)
    for asset in asset_data['AllResults']:
        insert_to_db(asset, city)

    return asset_data["IsLastPage"]


def get_asset_data(page_num, city_payload):
    city_payload['PageNo'] = page_num
    response = requests.post(url=URL, data=city_payload)
    asset_data = json.loads(response.text)
    return asset_data